# <center> Проект ML-инженера
## <center> РПодготовка рекомендательной системы

Импорт библиотек

In [35]:
pip install lightfm-dataset-helper

Note: you may need to restart the kernel to use updated packages.Collecting lightfm-dataset-helper



In [36]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
import pickle

ModuleNotFoundError: No module named 'lightfm'

Чтение данных

In [19]:
ratings = pd.read_csv('data/ratings.csv')
books = pd.read_csv('data/books.csv')
tags = pd.read_csv('data/tags_cleaned.csv')
book_tags = pd.read_csv('data/book_tags.csv')

In [27]:
display(ratings.head(1))
display(books.head(1))
display(tags.head(1))
display(book_tags.head(1))

,user_id,book_id,rating
0,1,258,5


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


,tag_id,tag_name
0,509,19th-century


,goodreads_book_id,tag_id,count,id
1,1,11305,37174,27


In [22]:
# Словарь, с помощью которого можно находить book_id книги по goodreads_book_id
mapper = dict(zip(books.goodreads_book_id,books.book_id))
# добавим id книги в датафрейм book_tags
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])

In [23]:
book_tags.head()

,goodreads_book_id,tag_id,count,id
1,1,11305,37174,27
4,1,33114,12716,27
5,1,11743,9954,27
6,1,14017,7169,27
10,1,27199,3857,27


In [24]:
# для работы с моделями в LightFM создаём разрежённые матрицы
#  будем хранить данные в формате COO (Coordinate List)
ratings_coo = sparse.coo_matrix((ratings.rating,(ratings.user_id, ratings.book_id)))
feature_ratings  = sparse.coo_matrix(([1]*len(book_tags), (book_tags.id, book_tags.tag_id)))

### Посторение модели

Объявим вспомогательные константы для обучения модели:

In [28]:
# число потоков процессора
NUM_THREADS = 1 # для Windows
# число параметров вектора 
NUM_COMPONENTS = 60
# число эпох обучения
NUM_EPOCHS = 10 
# зерно датчика случайных чисел
RANDOM_STATE = 42

Используем библиотеку LightFM, чтобы сделать матричное разложение (ALS) рейтингов книг и получить два набора векторов

In [29]:
# Разбиваем на обучающую и тестовую выборки
train, test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=RANDOM_STATE)

# Создаём модель
model = LightFM(
    learning_rate=0.05, # темп (скорость) обучения
    loss='warp', # loss-функция
    no_components=NUM_COMPONENTS, # размерность вектора признаков
    random_state=RANDOM_STATE # генератор случайных чисел
)

# Обучаем модель
model = model.fit(
    train, # обучающая выборка
    epochs=NUM_EPOCHS, # количество эпох обучения
    num_threads=NUM_THREADS, # количество потоков процессора
    item_features=feature_ratings # признаки товаров (рейтинги книг)
)

NameError: name 'random_train_test_split' is not defined

Тестирование модели

In [ ]:

precision_score = precision_at_k(
    model, # модель
    test, # тестовая выборка
    num_threads=NUM_THREADS, # количество потоков процессора
    k=10, # количество предложений
    item_features=feature_ratings # признаки товаров
).mean() # усредняем результаты
 
recall_score = recall_at_k(
    model, # модель
    test, # тестовая выборка
    num_threads=NUM_THREADS, # количество потоков процессора
    k=10, # количество предложений
    item_features=feature_ratings # признаки товаров
).mean() # усредняем результаты

print(recall_score, precision_score)

Сохраним модель

In [ ]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

## Добавим эмбеддинги к модели и посмотрим, что получилось

In [ ]:
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
# Достаём эбмеддинги
item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

print(item_biases.shape, item_embeddings.shape)

In [ ]:
#!pip install nmslib

In [ ]:
import nmslib

In [ ]:
#Инициализируем наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [ ]:
#Вспомогательная функция для поиска по графу
def nearest_books_nms(book_id, index, n=10):
    nn = index.knnQuery(item_embeddings[book_id], k=n)
    return nn

Найдем id книги 1984

In [ ]:
#Отфильтруем только те, где в названии встречается подстрока "1984"
books[books['title'].apply(lambda x: x.lower().find('1984')) >= 0]

Теперь найдем все похожие книги и посмотрим на них

In [ ]:
#Вызываем функцию для поиска ближайших соседей
print(nearest_books_nms(846, nms_idx))

In [ ]:
#Выделяем идентификаторы рекомендованных книг
nbm = nearest_books_nms(846, nms_idx)[0]
nbm

In [ ]:
#Посмотрим на авторов и названия рекомендованных книг
books[books.book_id.isin(nbm)][['authors', 'title']]

Сохраним эмбеддинги

In [ ]:
with open('item_embeddings.pkl', 'wb') as file:
    pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)